In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Yashar\\End-to-End-Employee-Classification-with-MLOPs\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Yashar\\End-to-End-Employee-Classification-with-MLOPs'

## config_entity.py

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

## configuration.py

In [16]:
# Update Configuration

from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path
        )

        return data_transformation_config
    


## components/data_transformation.py

In [20]:
import os
from mlProject import logger
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import joblib
import pandas as pd

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def data_preparation(self):
        data = pd.read_csv(self.config.data_path)
        data['Tenure'] = 2025 - data['JoiningYear']

        X = data.drop("LeaveOrNot", axis=1)
        y = data["LeaveOrNot"]

        logger.info("splitting the data into dependet and independent variables")
        logger.info(X.shape)
        logger.info(y.shape)

        print(X.shape)
        print(y.shape)

        # Define preprocessor
        categorical_nominal = ["City", "Gender", "EverBenched"]
        categorical_ordinal = ["Education"]
        numerical_cols = ["JoiningYear", "PaymentTier", "Age", "ExperienceInCurrentDomain", "Tenure"]
        preprocessor = ColumnTransformer(
            transformers=[
                ("cat_nominal", OneHotEncoder(handle_unknown="ignore"), categorical_nominal),
                ("cat_ordinal", OrdinalEncoder(categories=[["Bachelors", "Masters", "PHD"]]), categorical_ordinal),
                ("num", "passthrough", numerical_cols)
            ]
        )

        # Fit preprocessor (to be used in training)
        preprocessor.fit(X)

        # Save preprocessor
        joblib.dump(preprocessor, os.path.join(self.config.root_dir,'preprocessor.pkl'))
        logger.info("the Preprocessor is saved successfully! ")

    


## pipeline/stage_03_data_transformation.py

In [21]:
# Updating the Pipeline

try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.data_preparation()
except Exception as e:
    raise e


[2025-07-14 18:41:16,280: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-14 18:41:16,281: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-14 18:41:16,283: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-14 18:41:16,283: INFO: common: created directory at: artifacts]
[2025-07-14 18:41:16,283: INFO: common: created directory at: artifacts/data_transformation]
[2025-07-14 18:41:16,293: INFO: 901742189: splitting the data into dependet and independent variables]
[2025-07-14 18:41:16,294: INFO: 901742189: (4653, 9)]
[2025-07-14 18:41:16,295: INFO: 901742189: (4653,)]
(4653, 9)
(4653,)
[2025-07-14 18:41:16,309: INFO: 901742189: the Preprocessor is saved successfully! ]
